### Run the model on 5 subjects

In [1]:
from imagery_psychophysics.src.variational import *
from imagery_psychophysics.utils.variational_utils import open_imagery_probe_data
from imagery_psychophysics.src.stirling_maps import stirling_num_of_2nd_kind as snk
from imagery_psychophysics.src.stirling_maps import stirling_partitions
from skimage.morphology import binary_dilation, binary_erosion
import numpy as np
import pandas as pd
import math
import copy
from matplotlib import pyplot as plt
from os.path import join
from PIL.Image import open as open_image
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
/tmp/try_flags_xfXQUo.c:4:19: fatal error: cudnn.h: No such file or directory
compilation terminated.

Mapped name None to device cuda: GeForce GTX TITAN X (0000:01:00.0)


In [2]:
resultsDict = dict()
experiment_df = open_imagery_probe_data()

In [3]:
experiment_df['image'].unique()

array(['candle_01', 'peaches_05', 'cheeseburger_03', 'grape_juice_02'],
      dtype=object)

In [4]:
for subject, subject_group in experiment_df.groupby('subj'):
    resultsDict[subject] = dict()
    for state, state_group in subject_group.groupby('state'):
        resultsDict[subject][state] = dict()
        for targetImageName, target_group in state_group.groupby('image'):
            ##get data
            windows, resp, _, targetObjectMap,targetImage = open_imagery_probe_data(subject, state, targetImageName)
            targetObjectMap = targetObjectMap.resize(targetImage.size) ##have to do this because not matched in size
            
            ##==========construct model random variables
            ##number of objects
            nObj = numObjects()

            ##dispersion on category prior: here we set a hyperprior
            pDisp = priorDispersion()
            dispersion = 1.0
            pDisp.set_value(dispersion)

            ##resolution of object map Z
            nPixels = numPixels()

            ##category prior and object map
            catProb = categoryProbs(nObj,pDisp)
            Z = latentObjMap(catProb,nPixels)

            ##noise params
            nP = noiseParams()

            ##windows: we change their shape a little to make them easier to work with
            desiredWindowShape = (375,600)
            workingScale = 5 ##(50,80)
            w = probes()
            ##this whole "resolutions" thing doesn't work!! not a bug, just 90% useless as a tool for selecting
            ##a reasonable resolution for the model
            resolutions, workingResolution = w.resolve(desiredWindowShape, workingScale)
            w.set_value(w.reshape(windows, workingResolution),flatten=True)
            print 'working resolution is (%d, %d)' %(workingResolution[0], workingResolution[1])


            ##response object
            r = responses(Z,nP)

            ##real data
            r.set_values(windows=w)

            r.set_values(data=resp)

            print 'total observations: %d' %(r.N)

            tMap = target_image()
            targetObjectMap_test, targetImage_test = tMap.reshape(targetObjectMap,workingResolution,targetImage=targetImage)
            tMap.set_values(targetObjectMap_test, targetImage_test)
            
            ##==gonna cheat here and set the number of objects using target map
            thisK = tMap.numberTargetObjects
            r.Z.categoryPrior.numObjects.set_value(tMap.numberTargetObjects)
            
            ##==fixing resolution at some reasonble thing
            ##==resolutions[1] = (10 x 16)
            thisNumPix = (resolutions[1][0], resolutions[1][1])

            r.Z.numPixels.set_value(thisNumPix[0], thisNumPix[1])
            
            
            ##=============
            
            ##instantiate the variational inferences we want to perform
            iqZ = inferQZ()
            iqPi = inferQPi()

            ##...and the parameter optimizations (point-estimate) we want
            oNP = optimizeNoiseParams()

            ##variational inference combines them all together
            vi = VI(r, iqZ,oNP, iqPi)

            ### Run variational inference

            ##inference algorithm parameters
            initialNoisinessOfZ = 0.2
            pOn_init, pOff_init = .8, 0.2
            densityOfNoiseParamGrid = 50
            numStarterMaps = 20
            numSamplesForComputingObjectCountProbs = 20
            maxNumIterations = 300
            trainTestSplit = 0.8
            trainRegSplit = .8

            print '=========================(subject, state, target) = (%s, %s, %s) ====' %(subject, state, targetImageName)
            print '=========================(number of object = %d, number of pixels = (%d,%d) )' %(thisK, thisNumPix[0],thisNumPix[1])
            bestModel,_ = vi.run_VI(initialNoisinessOfZ, \
                                 pOn_init, pOff_init, \
                                 densityOfNoiseParamGrid, \
                                 numStarterMaps, \
                                 numSamplesForComputingObjectCountProbs, \
                                 maxNumIterations, \
                                 trainTestSplit, trainRegSplit, \
                                 optimizeHyperParams=False)

            resultsDict[subject][state][targetImageName]= copy.deepcopy(bestModel)

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


working resolution is (50, 80)
total observations: 345
=========================(subject, state, target) = (AS, img, candle_01) ====
=========================(number of object = 4, number of pixels = (10,16) )


/usr/local/lib/python2.7/dist-packages/theano/tensor/subtensor.py:2320: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


!new best!
ELBO: -312.856812
goodness of fit: -404.994324
posterior_entropy: 92.137497
mean log of predictive distribution over test samples: -1.868378
percent correct over test samples: 23.636364


!new best!
ELBO: -107.015305
goodness of fit: -253.605392
posterior_entropy: 146.590088
mean log of predictive distribution over test samples: -1.141762
percent correct over test samples: 54.545455


ELBO: -86.128922
goodness of fit: -226.102524
posterior_entropy: 139.973602
mean log of predictive distribution over test samples: -1.064784
percent correct over test samples: 50.909091


!new best!
ELBO: -88.952377
goodness of fit: -212.285706
posterior_entropy: 123.333328
mean log of predictive distribution over test samples: -0.990589
percent correct over test samples: 63.636364


!new best!
ELBO: -96.135406
goodness of fit: -199.753815
posterior_entropy: 103.618408
mean log of predictive distribution over test samples: -0.965397
percent correct over test samples: 65.454545


ELBO: -101.9150

ELBO: -97.372375
goodness of fit: -118.845840
posterior_entropy: 21.473465
mean log of predictive distribution over test samples: -0.787546
percent correct over test samples: 72.727273


ELBO: -97.609055
goodness of fit: -119.064423
posterior_entropy: 21.455370
mean log of predictive distribution over test samples: -0.772779
percent correct over test samples: 72.727273


ELBO: -96.859924
goodness of fit: -118.351135
posterior_entropy: 21.491209
mean log of predictive distribution over test samples: -0.788556
percent correct over test samples: 72.727273


ELBO: -98.685135
goodness of fit: -119.989143
posterior_entropy: 21.304008
mean log of predictive distribution over test samples: -0.790182
percent correct over test samples: 72.727273


ELBO: -98.145393
goodness of fit: -119.627663
posterior_entropy: 21.482267
mean log of predictive distribution over test samples: -0.788487
percent correct over test samples: 72.727273


ELBO: -97.406364
goodness of fit: -118.764015
posterior_entropy: 

...

ELBO: -110.367302
goodness of fit: -170.601730
posterior_entropy: 60.234428
mean log of predictive distribution over test samples: -0.703745
percent correct over test samples: 69.090909


ELBO: -110.373596
goodness of fit: -170.908173
posterior_entropy: 60.534576
mean log of predictive distribution over test samples: -0.701853
percent correct over test samples: 70.909091


ELBO: -112.396767
goodness of fit: -172.865540
posterior_entropy: 60.468773
mean log of predictive distribution over test samples: -0.702094
percent correct over test samples: 67.272727


ELBO: -107.290627
goodness of fit: -168.517319
posterior_entropy: 61.226692
mean log of predictive distribution over test samples: -0.724270
percent correct over test samples: 67.272727


ELBO: -108.329651
goodness of fit: -169.735336
posterior_entropy: 61.405685
mean log of predictive distribution over test samples: -0.706578
percent correct over test samples: 69.090909


ELBO: -110.576157
goodness of fit: -171.410843
posterior_ent

ELBO: -108.900482
goodness of fit: -169.777771
posterior_entropy: 60.877285
mean log of predictive distribution over test samples: -0.711121
percent correct over test samples: 69.090909


ELBO: -108.920731
goodness of fit: -170.917099
posterior_entropy: 61.996372
mean log of predictive distribution over test samples: -0.705367
percent correct over test samples: 69.090909


ELBO: -111.301910
goodness of fit: -172.545670
posterior_entropy: 61.243759
mean log of predictive distribution over test samples: -0.709278
percent correct over test samples: 67.272727


ELBO: -108.918701
goodness of fit: -169.617432
posterior_entropy: 60.698727
mean log of predictive distribution over test samples: -0.711176
percent correct over test samples: 67.272727


ELBO: -107.082474
goodness of fit: -168.706863
posterior_entropy: 61.624390
mean log of predictive distribution over test samples: -0.693805
percent correct over test samples: 69.090909


ELBO: -110.774490
goodness of fit: -173.315552
posterior_ent

ELBO: -109.754852
goodness of fit: -170.152512
posterior_entropy: 60.397659
mean log of predictive distribution over test samples: -0.707452
percent correct over test samples: 67.272727


ELBO: -109.390953
goodness of fit: -170.643555
posterior_entropy: 61.252602
mean log of predictive distribution over test samples: -0.721781
percent correct over test samples: 69.090909


ELBO: -109.043480
goodness of fit: -168.461746
posterior_entropy: 59.418266
mean log of predictive distribution over test samples: -0.698844
percent correct over test samples: 69.090909


ELBO: -110.739090
goodness of fit: -171.489410
posterior_entropy: 60.750324
mean log of predictive distribution over test samples: -0.685806
percent correct over test samples: 67.272727


ELBO: -110.194969
goodness of fit: -171.759537
posterior_entropy: 61.564568
mean log of predictive distribution over test samples: -0.694103
percent correct over test samples: 69.090909


ELBO: -110.712486
goodness of fit: -170.414139
posterior_ent

ELBO: -109.229874
goodness of fit: -171.043457
posterior_entropy: 61.813583
mean log of predictive distribution over test samples: -0.688915
percent correct over test samples: 67.272727


ELBO: -110.973282
goodness of fit: -172.841187
posterior_entropy: 61.867905
mean log of predictive distribution over test samples: -0.721903
percent correct over test samples: 67.272727


ELBO: -110.075905
goodness of fit: -170.594742
posterior_entropy: 60.518837
mean log of predictive distribution over test samples: -0.704071
percent correct over test samples: 69.090909


ELBO: -112.685127
goodness of fit: -173.673676
posterior_entropy: 60.988548
mean log of predictive distribution over test samples: -0.714221
percent correct over test samples: 67.272727


ELBO: -109.384636
goodness of fit: -169.676025
posterior_entropy: 60.291389
mean log of predictive distribution over test samples: -0.705611
percent correct over test samples: 67.272727


ELBO: -108.755295
goodness of fit: -168.383560
posterior_ent

ELBO: -110.998871
goodness of fit: -171.929840
posterior_entropy: 60.930969
mean log of predictive distribution over test samples: -0.681651
percent correct over test samples: 67.272727


ELBO: -110.023979
goodness of fit: -170.531509
posterior_entropy: 60.507530
mean log of predictive distribution over test samples: -0.683786
percent correct over test samples: 67.272727


ELBO: -110.368256
goodness of fit: -170.501404
posterior_entropy: 60.133148
mean log of predictive distribution over test samples: -0.709981
percent correct over test samples: 70.909091


ELBO: -109.020676
goodness of fit: -168.095657
posterior_entropy: 59.074982
mean log of predictive distribution over test samples: -0.677533
percent correct over test samples: 69.090909


ELBO: -110.411766
goodness of fit: -169.800385
posterior_entropy: 59.388618
mean log of predictive distribution over test samples: -0.697845
percent correct over test samples: 69.090909


ELBO: -110.286255
goodness of fit: -169.071121
posterior_ent

#### Store the results

In [5]:
modelList = []
for sbj,subjectDict in resultsDict.iteritems():
    for st, stateDict in subjectDict.iteritems():
        for targ, modelInstance in stateDict.iteritems():
            modelList.append([sbj,st,targ, modelInstance])

In [6]:
modelDf  = pd.DataFrame(data=modelList, columns=['subject', 'state', 'target', 'model'])
modelDf

,subject,state,target,model
0,gsy,pcp,candle_01,<imagery_psychophysics.src.variational.VI obje...
1,gsy,pcp,peaches_05,<imagery_psychophysics.src.variational.VI obje...
2,gsy,img,candle_01,<imagery_psychophysics.src.variational.VI obje...
3,gsy,img,peaches_05,<imagery_psychophysics.src.variational.VI obje...
4,AS,pcp,candle_01,<imagery_psychophysics.src.variational.VI obje...
5,AS,pcp,peaches_05,<imagery_psychophysics.src.variational.VI obje...
6,AS,img,candle_01,<imagery_psychophysics.src.variational.VI obje...
7,AS,img,peaches_05,<imagery_psychophysics.src.variational.VI obje...
8,ltd,pcp,candle_01,<imagery_psychophysics.src.variational.VI obje...
9,ltd,pcp,peaches_05,<imagery_psychophysics.src.variational.VI obje...


In [7]:
store = pd.io.pytables.HDFStore('/home/tnaselar/FAST/scratch/five_subjects_(10x16).h5')
##after saving to scratch I move it to imagery_psychophysic/multipolyprobe/data

In [8]:
store['models'] = modelDf

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2882: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['subject', 'state', 'target', 'model']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
store.close()